In [219]:
import pandas as pd

In [ ]:
data = pd.read_csv("test.csv", header=None)
data.head()

### Rename columns

In [ ]:
def clean_data():
    # rename columns
    data.columns = ['datetime', 'event']
    # filter data 
    data = data[(data.datetime != "time") | (data.event != "event") ]
    data.event = data.event.str.lower()

    # covert datetime column to datetime object. SHOULD BE DONE AFTER FILTERING. ELSE, ERRORS
    data.datetime = pd.to_datetime(data.datetime)

    data["month"] = data.datetime.dt.month_name()
    data["time"] = data.datetime.dt.time

In [ ]:
data.columns = ['datetime', 'event']
data.head()

In [222]:
data = data[(data.datetime != "time") | (data.event != "event") ]

In [ ]:
data.iloc[15980]

In [ ]:
#data.datetime = pd.to_datetime(data.iloc[:,0])
data.datetime = pd.to_datetime(data.datetime)
data.dtypes


In [ ]:
data["month"] = data.datetime.dt.month_name()
data["date"] = data.datetime.dt.time
data.head()

In [ ]:
data.event = data.event.str.lower()
data.head()

In [227]:
#data.to_csv("clean_data.csv")

In [228]:
classify_list = ["station supply fail","trip cct fault","dc power failure","cb lockout locked","cb backup trip",
"cb back prot trip","dc fault","homa station supply fail fault"]
def classify_events(row):  
    for s in classify_list:
        if s in row:
            return "critical"

        else:
            return "normal"


In [ ]:
#df = df.apply(lambda row : replace(row))
data['type'] = data['event'].apply(lambda row: classify_events(row))
data.head()

In [230]:
data.type.value_counts()

normal      15945
critical       60
Name: type, dtype: int64

In [231]:
# [i for e in bad for i in my_list if e in i]
# for i in check_list:
#     for e in classify_list:
#         if e in i:
#             my_list.append(i)

# my_list

In [232]:
ciritcal_data = data[data['type'] == 'critical']
ciritcal_data.shape

(60, 5)

In [61]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

load_dotenv()


True

In [62]:
host = os.getenv("HOST")
db_username = os.getenv("DB_USERNAME")
password = os.getenv("PASSWORD")
db_name = os.getenv("LOCAL_DB")
port = os.getenv("PORT")

In [63]:
engine = create_engine(f'mysql+pymysql://root:{password}@{host}/{db_name}')

In [ ]:
raw_data = pd.read_excel("SOE_4th_22nd_Nov_2022.xlsx", sheet_name=0, header=None)
raw_data.columns = ['datetime', 'event']
raw_data.head()

In [65]:
raw_data.to_sql(con=engine, name="scada_data", if_exists="append")

## Get raw data from database

In [ ]:
db_data = pd.read_sql("SELECT * FROM scada_data", con=engine)
db_data.head()

In [69]:
today_date = datetime.today().strftime('%Y-%m-%d')
print(today_date)
type(today_date)

2022-12-18


str

In [70]:
serach_date = datetime.strftime(datetime.now() - timedelta(25), '%Y-%m-%d')
print(serach_date)
type(serach_date)

2022-11-23


str

In [71]:
past_days = 37

In [72]:
sql_query = "SELECT * FROM scada_data WHERE DATE(datetime) = %(serach_date)s"
my_params={"serach_date":datetime.strftime(datetime.now() - timedelta(past_days), '%Y-%m-%d')}
filtered_data = pd.read_sql(sql_query,con=engine, params=my_params)

filtered_data.head()

,index,datetime,event
0,5267,2022-11-11 00:07:02,LIRA 33 BS CB Closed
1,5268,2022-11-11 00:18:25,MBRN 33 KBLE CB Main Prot Trip
2,5269,2022-11-11 00:18:25,MBRN 33 KBLE CB Lockout Locked
3,5270,2022-11-11 00:18:25,MBRN 33 KBLE CB Open
4,5271,2022-11-11 00:18:25,MBRN 33 KBLE CB Aux DC Fault


In [73]:
filtered_data.shape

(807, 3)

## store substation details in a db table

In [81]:
substation_data = pd.read_excel("substation details.xlsx", index_col="no",sheet_name=0)
substation_data.columns = substation_data.columns.str.lower().str.replace(" ","_")
#substation_data.columns
substation_data.to_sql(con=engine, name="substation_data", if_exists="append")

In [82]:
sql_qyery = "SELECT substation_acronym,critical_level,substation_name FROM substation_data"
substation_data = pd.read_sql(sql_qyery,con=engine)
substation_data.head()

,substation_acronym,critical_level,substation_name
0,mskw,3,masaka west
1,nkda,4,nkenda
2,owfs,5,owen falls
3,homa,3,hoima
4,ofex,5,owen falls extension
